# Treinamento de um Modelo para Classificação de Tweets com Discursos de Ódio

### Contém o código que rotula nosso dataset de avaliação, e verifica a acurácia para tweets customizados que criamos

Membros: Lucas de Medeiros Soares, João Henrique Almeida Xavier e Tarso Jabbes Lima de Oliveira

In [1]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm

/home/tarsojabbes/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Criação do Pipeline com o Modelo

In [2]:
PATH = "../model"
classifier = pipeline('text-classification', model= PATH, tokenizer= PATH)

In [3]:
df_all = pd.read_csv('../Dados/test_data2.csv')

In [4]:
df_all.head()

,query,label
0,"Assim, sim! Uma voz lúcida na cúpula da Igreja...",0
1,TODO ESQUERDISTA TEM UMA MENTE REDUCIONISTA ...,1
2,Não se sinta mulher de verdade se você ofende ...,1
3,Bue fufas tão a seguir no insta pq wtf,1
4,Não quer refugiados? Então pare de criá-los. h...,1


In [5]:
classifier(df_all['query'][0])

[{'label': 'LABEL_0', 'score': 0.9994864463806152}]

## Execução do classificador para o dataset de avaliação

In [6]:
# Cria uma lista vazia para armazenar as previsões filtradas
preds = []

# Itera sobre as queries (texto) no DataFrame df_all
# tqdm(df_all['query']): exibe uma barra de progresso enquanto itera sobre as queries
for i, query in enumerate(tqdm(df_all['query'])):
    try:
        # Aplica o classificador ao texto (query) e obtém a previsão (pred)
        pred = classifier(query)
        
        # Verifica se o rótulo previsto é 'LABEL_1' (discurso de ódio, por exemplo) e a confiança (score) é superior a 0.8
        if pred[0]['label'] == 'LABEL_1' and pred[0]['score'] > 0.8:
            # Se a condição for satisfeita, adiciona a query correspondente à lista 'preds'
            preds.append(df_all['query'][i])
    
    # Tratamento de exceções: se ocorrer um erro de tempo de execução (RuntimeError), imprime a query que causou o erro
    except RuntimeError as e:
        print(f"query: {query}")


100%|██████████| 1074/1074 [01:12<00:00, 14.80it/s]


In [7]:
len(preds)

424

In [8]:
# Cria uma nova coluna 'classification' no DataFrame df_all
# A função lambda verifica se cada 'query' está na lista 'preds'
# Se a query estiver na lista 'preds', atribui 1 (indica que foi classificada como LABEL_1, por exemplo)
# Caso contrário, atribui 0 (indica que a query não foi classificada como LABEL_1)
df_all['classification'] = df_all['query'].apply(lambda x: 1 if x in preds else 0)


In [9]:
df_all.to_csv('../Dados/results_data2.csv', index=False)

## Classificação para exemplos extras criados

In [10]:
# Carrega o DataFrame a partir de um arquivo CSV chamado 'custom.csv'
df_custom = pd.read_csv('../Dados/custom.csv')

# Inicializa uma lista vazia para armazenar as previsões filtradas
preds = []

# Itera sobre as queries no DataFrame df_custom, mostrando uma barra de progresso com tqdm
for i, query in enumerate(tqdm(df_custom['query'])):
    try:
        # Aplica o classificador ao texto da query e obtém as previsões
        pred = classifier(query)
        
        # Verifica se o rótulo da previsão é 'LABEL_1' (ex.: discurso de ódio) e a confiança é maior que 0.8
        if pred[0]['label'] == 'LABEL_1' and pred[0]['score'] > 0.8:
            # Adiciona a query à lista 'preds' se a condição for atendida
            preds.append(df_custom['query'][i])    
    except RuntimeError as e:
        # Tratamento de exceção: imprime a query que gerou um erro de execução (se ocorrer)
        print(f"query: {query}")

# Adiciona uma nova coluna 'classification' ao DataFrame
# Para cada query, a função lambda atribui 1 se ela estiver na lista 'preds', ou 0 se não estiver
df_custom['classification'] = df_custom['query'].apply(lambda x: 1 if x in preds else 0)

# Salva o DataFrame atualizado em um arquivo CSV chamado 'custom_result.csv' sem os índices
df_custom.to_csv('../Dados/custom_result.csv', index=False)


100%|██████████| 20/20 [00:00<00:00, 21.04it/s]


In [11]:
# Carrega o DataFrame contendo os resultados de classificação a partir do arquivo 'custom_result.csv'
df_custom_result = pd.read_csv("../Dados/custom_result.csv")

# Cria uma nova coluna 'acerto' no DataFrame, que armazena True para acertos (quando 'label' e 'classification' são iguais)
# e False para erros (quando 'label' e 'classification' são diferentes)
df_custom_result['acerto'] = df_custom_result['label'] == df_custom_result['classification']

# Conta o número de acertos somando os valores True na coluna 'acerto'
acertos = df_custom_result['acerto'].sum()

# Calcula o número de erros subtraindo o número de acertos do número total de exemplos no DataFrame
erros = len(df_custom_result) - acertos

# Exibe o número de acertos e erros
print(f"Acertos: {acertos}")
print(f"Erros: {erros}")

# Calcula a porcentagem de acertos (acertos / número total de exemplos * 100)
porcentagem_acertos = acertos / len(df_custom_result) * 100

# Exibe a porcentagem de acertos formatada com 2 casas decimais
print(f"Porcentagem de Acertos: {porcentagem_acertos:.2f}%")


Acertos: 19
Erros: 1
Porcentagem de Acertos: 95.00%
